# Data Analysis of Hacker News Website

This notebook analyzes posts on Hacker News. The data can be found [here](https://www.kaggle.com/datasets/hacker-news/hacker-news-posts). The data set contains one year of posts up to September 26, 2016.